## ○動的計画法（DP）による貪欲法を用いた複数ナップザック問題の近似解推定

In [1]:
import time
from ortoolpy import knapsack
import random
import numpy as np
import pandas as pd

In [2]:
# サンプルデータの読み込み
df_sample = pd.read_csv('sample_3*30.csv', sep='\t')

In [3]:
df_sample.head(3)

,id,price_0,price_1,price_2,rating_0,rating_1,rating_2
0,1,380,385,389,4.1,3.3,4.0
1,2,943,947,961,10.1,8.9,9.3
2,3,980,980,987,10.4,8.7,10.3


In [4]:
# clientの数。
client_num = int((len(df_sample.columns)-1)/2)
print('クライアントの数:', client_num)

クライアントの数: 3


In [5]:
# 初めにクライアントが持っていた枠をランダムに決める（別のデータに対応できるかを見るため。）
sample = df_sample.sample(frac=1).reset_index(drop=True)
sample.head(3)

,id,price_0,price_1,price_2,rating_0,rating_1,rating_2
0,14,1047,1058,1039,9.8,11.3,10.3
1,30,607,607,600,6.1,7.0,5.1
2,1,380,385,389,4.1,3.3,4.0


In [6]:
# クライアト0, 1, 2の元々持っていた枠をランダムに決める。
length = len(sample) # データフレームの長さ
bps = [0 for i in range(client_num+1)] # Break Points
bp = 0
original_prices = [0 for i in range(client_num)] # 元々持っていた枠の価格和。
original_points = [0 for i in range(client_num)] # 元々持っていた枠の視聴率和。

for i in range(client_num):
    bp += length//client_num
    bps[i+1] = bp
    df_original = sample[bps[i]:bps[i+1]]
    original_prices[i] = sum(df_original['price_' + str(i)]) # 価格の和。
    original_points[i] = sum(df_original['rating_' + str(i)]) # 視聴率の和。

In [7]:
print(original_prices) # 元々持っていた枠の価格和。
print(original_points) # 元々持っていた枠の視聴率和。

[6208, 5157, 7668]
[60.300000000000004, 51.8, 76.0]


***
***

### 〜ここからナップザック問題（DP貪欲法のアプローチ）のアルゴリズム〜

In [8]:
# 必要な視聴率和(=元々持っていた視聴率和)
require_ratings = list(original_points) # 参照してしまうことを避けるため。
print('注意！参照されてる！！' if id(require_ratings) == id(original_points) else '参照されてない。 good!!')

参照されてない。 good!!


In [9]:
# value=「視聴率/号数（価格）」を求める。(これを比較して、貪欲に取り出す。)
for i in range(client_num):
    df_sample['value_'+str(i)] = df_sample['rating_'+str(i)]/df_sample['price_'+str(i)]

# valueができていることの確認。
df_sample.head(3)

,id,price_0,price_1,price_2,rating_0,rating_1,rating_2,value_0,value_1,value_2
0,1,380,385,389,4.1,3.3,4.0,0.010789,0.008571,0.010283
1,2,943,947,961,10.1,8.9,9.3,0.010710,0.009398,0.009677
2,3,980,980,987,10.4,8.7,10.3,0.010612,0.008878,0.010436


<b><font color='Red'>※valueは、正の数じゃなくても良い（単なる大きさ比較だから。）</font>
    <br>しかし、余ったものをナップザック問題で解くので、そこでは正の数でないといけない。（０の方が大きいから。）
    <br>とはいえ、価格、視聴率のどちらかが負の数になることは考えにくいため、意識するる必要はなさげ。</b>

In [10]:
# 2重のリストをフラットにする関数(重複は残る！)
def Flatten_dual(nested_list):
    return [e for inner_list in nested_list for e in inner_list]

In [11]:
def DPMKP(df, original_prices, original_points, rate, client_num):
    result = dict() #結果を格納する。
    obtained_prices = [0 for i in range(client_num)] #獲得した枠の値段をそれぞれ格納する。
    obtained_points = [0 for i in range(client_num)] #獲得した枠の視聴率をそれぞれ格納する。
    obtained_idses = [[] for i in range(client_num)] #獲得した枠のidをそれぞれ格納する。
    idses = [[] for i in range(client_num)] # 各ナップザック問題でそれぞれが指定した枠のidを格納する。
    add_idses = [[] for i in range(client_num)] # 各ナップザック問題の結果、獲得した枠のidを格納する。
    count = 0
    
    while len(df)>0:
        for i in range(client_num):
            original_price = original_prices[i] # 交換前の価格合計。
            original_point = original_points[i] # 交換前の視聴率合計。
            size = list(np.array(df['price_' + str(i)]))
            weight = list(np.array(df['rating_' + str(i)]))
            capacity = original_price*rate - obtained_prices[i]
            idses[i] = knapsack(size, weight, capacity)[1]

        ids_dict = dict()
        for key in range(len(df)): # それぞれのindex
            values = [] # そのidの枠を指定したクライアントを格納するリスト。
            for j in range(client_num): # それぞれのクライアントごとに、
                if key in idses[j]: # そのid（のindex）があれば、
                    values.append(j) # valuesにクライアントの名前が格納される。
            if values: # そのidを指定したクライアントがいれば、
                value = random.choice(values) # その中からクライアントをランダムに選ぶ。
                ids_dict[key] = value # idと合わせて格納される。

        if len(ids_dict)==0: # もしどのクライアントも選ばなかったら、
            rate += 0.01*(count) # 仕方なくrateを少しずつ上げていく。
            count += 1
            
        for i in range(client_num):
            add_idses[i] = [ids for ids, client in ids_dict.items() if client == i] # クライアントごとに、獲得したid
            # obtained_idses[i] += add_idses[i]
            obtained_idses[i] += df.query('index in ' + str(add_idses[i]))['id'].values.tolist()
            obtained_prices[i] += sum(df.query('index in ' + str(add_idses[i]))['price_' + str(i)])
            obtained_points[i] += sum(df.query('index in ' + str(add_idses[i]))['rating_' + str(i)])

        extraction_ids = Flatten_dual(add_idses) # 今回獲得されたidの集合を、取り除く。
        df = df.query('index not in ' + str(extraction_ids)).reset_index(drop=True)
    
    if obtained_points > original_points:
        sum_point = round(sum(obtained_points), 2) # 合計値を小数第二位までで表示。
        result[sum_point] = obtained_prices, obtained_idses
    
    return result

In [12]:
# 1回のループの結果。
start = time.time()
result = DPMKP(df_sample, original_prices, original_points, 1.05, client_num)
end = time.time()
print('処理時間は:', round(end-start, 3), '[sec]')
result

処理時間は: 0.234 [sec]


{205.0: ([6144, 4812, 8007],
  [[1, 2, 4, 6, 7, 12, 23, 26, 29, 24],
   [8, 9, 11, 13, 16, 20, 27, 30],
   [3, 5, 10, 15, 17, 18, 19, 21, 22, 25, 28, 14]])}

In [13]:
# 10回分のループ処理時間。
start = time.time()

results = {}
for i in range(10):
    results.update(DPMKP(df_sample, original_prices, original_points, 1.05, client_num))
    
end = time.time()
print('10回分の処理時間は:', round(end-start, 3), '[sec]')

10回分の処理時間は: 2.292 [sec]


In [14]:
total_prices = [0 for i in range(client_num)]
total_points = [0 for i in range(client_num)]

for i in range(client_num):
    chose_ids = results[max(results.keys())][1][i]
    total_prices[i] += sum(df_sample.query('index in ' + str(chose_ids))['price_' + str(i)])
    total_points[i] += sum(df_sample.query('index in ' + str(chose_ids))['rating_' + str(i)])

In [15]:
print('交換前の合計価格　:', original_prices)
print('獲得枠の合計価格　:', total_prices)
print('交換前の合計視聴率:', np.round(np.array(original_points), 2))
print('獲得枠の合計視聴率:', np.round(np.array(total_points), 2))

交換前の合計価格　: [6208, 5157, 7668]
獲得枠の合計価格　: [6221, 6370, 5998]
交換前の合計視聴率: [60.3 51.8 76. ]
獲得枠の合計視聴率: [62.8 63.4 60. ]


### ○これだと、枠の数が少ない場合に交換前の視聴率和を越えない場合がある。